# Build XGB using GNN embeddings
In this notebook we build a simple GNN model and combine it with XGB model to see if GNN improves XGB

# Simple GNN Baseline

In [1]:
import os
os.environ.setdefault("CUDA_VISIBLE_DEVICES", "7")

LOAD = 2
VER=7

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F

from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

from torch_geometric.data import Data
from torch_geometric.nn import SAGEConv

In [3]:
author_features = pd.read_parquet(f"data/author_features_v{LOAD}.pqt")
author_targets = pd.read_parquet(f"data/author_targets_v{LOAD}.pqt")

In [4]:
author_targets.head()

,Author_ID,target,coauthor_ids,coauthor_counts,degree
0,0,0,[],[],0
1,1,0,"[2, 3, 4, 5, 6, 7, 8]","[1, 1, 1, 1, 1, 1, 1]",7
2,2,0,"[1, 3, 4, 5, 6, 7, 8]","[1, 1, 1, 1, 1, 1, 1]",7
3,3,0,"[1, 2, 4, 5, 6, 7, 8]","[1, 1, 1, 1, 1, 1, 1]",7
4,4,1,"[1, 2, 3, 5, 6, 7, 8, 2905, 2819, 2906, 2907, ...","[1, 1, 1, 4, 4, 1, 1, 2, 2, 1, 2, 2, 1, 1, 1, ...",38


In [5]:
N = len(author_features)
assert author_features["Author_ID"].is_unique

id2idx = pd.Series(np.arange(N), index=author_features["Author_ID"].values)

In [6]:
# LOG TRANSFORM FOR NN
for c in [
    "n_coauthors",
    "total_collaborations",
    "total_papers",
    "total_citations",
    "citations_last_3y",
    "max_citations_single_paper",
]:
    author_features[f"log_{c}"] = np.log1p(author_features[c])

In [7]:
TARGET = "target"

BASE_FEATURES = [
    # --------------------
    # Collaboration graph
    # --------------------
    "n_coauthors",
    "total_collaborations",
    "avg_collab_strength",
    "max_collab_strength",
    "collab_entropy",

    # --------------------
    # Productivity / recency
    # --------------------
    "total_papers",
    "papers_last_1y",
    "papers_last_3y",
    "days_since_last_paper",

    # --------------------
    # Topic specialization
    # --------------------
    #"top_category",          # categorical-as-ordinal (needs encoding for NN)
    "top_category_frac",
    "category_entropy",

    # --------------------
    # Citation impact
    # --------------------
    "total_citations",
    "avg_citations_per_paper",
    "max_citations_single_paper",
    "citations_last_3y",
]

EXTRA_FEATURES = [
    "log_n_coauthors",
    "log_total_collaborations",
    "log_total_papers",
    "log_total_citations",
    "log_citations_last_3y",
    "log_max_citations_single_paper",
]

FEATURES = [c for c in (BASE_FEATURES + EXTRA_FEATURES) if c in author_features.columns]
print("Using features:", FEATURES)

Using features: ['n_coauthors', 'total_collaborations', 'avg_collab_strength', 'max_collab_strength', 'collab_entropy', 'total_papers', 'papers_last_1y', 'papers_last_3y', 'days_since_last_paper', 'top_category_frac', 'category_entropy', 'total_citations', 'avg_citations_per_paper', 'max_citations_single_paper', 'citations_last_3y', 'log_n_coauthors', 'log_total_collaborations', 'log_total_papers', 'log_total_citations', 'log_citations_last_3y', 'log_max_citations_single_paper']


In [8]:
X = author_features[FEATURES].astype(np.float32).values
y = author_features[TARGET].astype(np.int64).values

# standardize
X_mean = X.mean(axis=0, keepdims=True)
X_std  = X.std(axis=0, keepdims=True) + 1e-6
X = (X - X_mean) / X_std

x = torch.tensor(X, dtype=torch.float)
y = torch.tensor(y, dtype=torch.long)

In [9]:
# Make sure coauthor lists align to author_features order
author_targets_aligned = author_targets.set_index("Author_ID").loc[author_features["Author_ID"]].reset_index()

src_list = []
dst_list = []
w_list   = []

for i, (co_ids, co_cts) in enumerate(
    zip(author_targets_aligned["coauthor_ids"], author_targets_aligned["coauthor_counts"])
):
    # skip isolated authors
    if len(co_ids) == 0:
        continue

    js = id2idx.loc[co_ids].values.astype(np.int64)
    cts = np.asarray(co_cts, dtype=np.float32)

    src_list.append(np.full(len(js), i, dtype=np.int64))
    dst_list.append(js)
    w_list.append(cts)

src = np.concatenate(src_list) if src_list else np.empty(0, dtype=np.int64)
dst = np.concatenate(dst_list) if dst_list else np.empty(0, dtype=np.int64)
w   = np.concatenate(w_list)   if w_list   else np.empty(0, dtype=np.float32)

edge_index = torch.tensor(np.vstack([src, dst]), dtype=torch.long)
edge_weight = torch.tensor(w, dtype=torch.float)

In [10]:
print("Num nodes:", N)
print("Num edges (directed):", edge_index.size(1))

Num nodes: 143691
Num edges (directed): 11182120


In [11]:
data = Data(x=x, edge_index=edge_index, y=y)
data.edge_weight = edge_weight  # optional

In [12]:
class GraphSAGE(torch.nn.Module):
    def __init__(self, in_dim, hidden_dim=128, dropout=0.1):
        super().__init__()
        self.conv1 = SAGEConv(in_dim, hidden_dim)
        self.conv2 = SAGEConv(hidden_dim, hidden_dim)
        self.lin   = torch.nn.Linear(hidden_dim, 1)
        self.dropout = dropout

    def forward(self, x, edge_index, return_emb=False):
        h = self.conv1(x, edge_index)
        h = F.relu(h)
        h = F.dropout(h, p=self.dropout, training=self.training)

        h = self.conv2(h, edge_index)
        h = F.relu(h)
        h = F.dropout(h, p=self.dropout, training=self.training)

        if return_emb:
            return h

        out = self.lin(h).squeeze(-1)
        return out

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data = data.to(device)

N_SPLITS = 5
kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=42)

N = data.num_nodes
EMB_DIM = 128

# OOF storage (optional, for diagnostics)
oof_gnn = np.zeros(N, dtype=np.float32)
oof_embs  = np.zeros((N, EMB_DIM), dtype=np.float32)

fold_aucs   = []
fold_states = []        
fold_splits = []        

def train_one_fold(train_idx, val_idx, epochs=300, lr=5e-3, wd=1e-4):
    model = GraphSAGE(
        in_dim=data.x.size(1),
        hidden_dim=EMB_DIM,
        dropout=0.1
    ).to(device)

    opt = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)

    train_mask = torch.zeros(N, dtype=torch.bool, device=device)
    val_mask   = torch.zeros(N, dtype=torch.bool, device=device)
    train_mask[torch.tensor(train_idx, device=device)] = True
    val_mask[torch.tensor(val_idx, device=device)] = True

    y_float = data.y.float()

    best_auc = -1.0
    best_state = None

    for epoch in range(1, epochs + 1):
        model.train()
        opt.zero_grad()

        logits = model(data.x, data.edge_index)
        loss = F.binary_cross_entropy_with_logits(
            logits[train_mask],
            y_float[train_mask]
        )

        loss.backward()
        opt.step()

        # validation AUC
        model.eval()
        with torch.no_grad():
            val_probs = torch.sigmoid(logits[val_mask]).cpu().numpy()
            val_true  = data.y[val_mask].cpu().numpy()
            auc = roc_auc_score(val_true, val_probs)

        if auc > best_auc:
            best_auc = auc
            best_state = {
                k: v.detach().cpu().clone()
                for k, v in model.state_dict().items()
            }

        if epoch % 30 == 0 or epoch == 1:
            print(
                f"  epoch {epoch:3d} | "
                f"loss {loss.item():.4f} | "
                f"val AUC {auc:.5f} | "
                f"best {best_auc:.5f}"
            )

    # restore best model
    model.load_state_dict({k: v.to(device) for k, v in best_state.items()})
    model.eval()

    with torch.no_grad():
        logits = model(data.x, data.edge_index)
        val_probs = torch.sigmoid(logits[val_mask]).cpu().numpy()

        emb = model(data.x, data.edge_index, return_emb=True)
        val_emb = emb[val_mask].cpu().numpy()

    return best_auc, best_state, val_probs, val_emb

In [14]:
for fold, (train_idx, val_idx) in enumerate(kf.split(np.arange(N)), 1):
    print(f"\n##### FOLD {fold} #####")

    auc, best_state, val_probs, val_emb = train_one_fold(
        train_idx,
        val_idx,
        epochs=300,
        lr=5e-3,
        wd=1e-4
    )

    fold_aucs.append(auc)
    fold_states.append(best_state)        # <-- SAVE MODEL
    fold_splits.append((train_idx, val_idx))  # <-- SAVE SPLIT

    oof_gnn[val_idx] = val_probs
    oof_embs[val_idx]  = val_emb

    print(f"Fold {fold} best AUC: {auc:.5f}")

print("\n==============================")
print("Fold AUCs:", [f"{a:.5f}" for a in fold_aucs])
print("Mean AUC :", float(np.mean(fold_aucs)))
print("OOF AUC  :", float(roc_auc_score(data.y.cpu().numpy(), oof_gnn)))
print("==============================")


##### FOLD 1 #####
  epoch   1 | loss 0.6937 | val AUC 0.56596 | best 0.56596
  epoch  30 | loss 0.5739 | val AUC 0.74487 | best 0.74658
  epoch  60 | loss 0.5698 | val AUC 0.75085 | best 0.75085
  epoch  90 | loss 0.5680 | val AUC 0.74948 | best 0.75173
  epoch 120 | loss 0.5655 | val AUC 0.75151 | best 0.75211
  epoch 150 | loss 0.5641 | val AUC 0.75170 | best 0.75228
  epoch 180 | loss 0.5626 | val AUC 0.75061 | best 0.75343
  epoch 210 | loss 0.5614 | val AUC 0.75155 | best 0.75343
  epoch 240 | loss 0.5603 | val AUC 0.75213 | best 0.75343
  epoch 270 | loss 0.5592 | val AUC 0.75101 | best 0.75343
  epoch 300 | loss 0.5596 | val AUC 0.75169 | best 0.75343
Fold 1 best AUC: 0.75343

##### FOLD 2 #####
  epoch   1 | loss 0.7109 | val AUC 0.46190 | best 0.46190
  epoch  30 | loss 0.5742 | val AUC 0.74590 | best 0.74590
  epoch  60 | loss 0.5695 | val AUC 0.74772 | best 0.75012
  epoch  90 | loss 0.5675 | val AUC 0.74937 | best 0.75097
  epoch 120 | loss 0.5653 | val AUC 0.75115 | best

In [15]:
# SAVE GNN OOF
np.save(f"data/oof_preds_gnn_v{VER}",oof_gnn)

# Train XGB w/ GNN Embeddings
We will now train XGB using GNN embeddings. This performs better than simple average of XGB and GNN probability predictions that was done in the previous notebook.

In [16]:
xgb_params = {
    "objective": "binary:logistic",
    "eval_metric": "auc",
    "learning_rate": 0.05,
    "max_depth": 6,
    "min_child_weight": 5,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "tree_method": "hist",
    "n_estimators": 1_000,
    "random_state": 42,
   "early_stopping_rounds": 50,
}

In [17]:
import xgboost as xgb
print(f"XGB version: {xgb.__version__}")

XGB version: 3.1.2


In [18]:
from scipy.stats import rankdata
from cuml.decomposition import PCA

# tabular features
X_tab = author_features[FEATURES].values.astype(np.float32)
y = author_features[TARGET].values.astype(np.int64)

N = len(y)
oof_preds = np.zeros(N, dtype=np.float32)
fold_scores = []

# IMPORTANT: reuse the SAME splits as GNN training
for fold, ((train_idx, val_idx), gnn_state) in enumerate(
    zip(fold_splits, fold_states), 1
):
    print(f"\n##### STACK FOLD {fold} #####")

    # -----------------------------
    # Load fold-specific GNN
    # -----------------------------
    gnn = GraphSAGE(
        in_dim=data.x.size(1),
        hidden_dim=EMB_DIM,
        dropout=0.1
    ).to(device)

    gnn.load_state_dict({k: v.to(device) for k, v in gnn_state.items()})
    gnn.eval()

    # -----------------------------
    # Extract embeddings (same space)
    # -----------------------------
    with torch.no_grad():
        emb_all = gnn(
            data.x,
            data.edge_index,
            return_emb=True
        ).cpu().numpy().astype(np.float32)

    # -----------------------------
    # PCA on embeddings (FIT = train only)
    # -----------------------------
    pca = PCA(n_components=8)
    emb_train_pca = pca.fit_transform(emb_all[train_idx])
    emb_val_pca   = pca.transform(emb_all[val_idx])

    # -----------------------------
    # Build stacked features
    # -----------------------------
    X_train = np.hstack([
        X_tab[train_idx],
        emb_train_pca
    ])

    X_val = np.hstack([
        X_tab[val_idx],
        emb_val_pca
    ])

    y_train, y_val = y[train_idx], y[val_idx]

    # -----------------------------
    # Train stack XGB (regularized!)
    # -----------------------------
    model = xgb.XGBClassifier(**xgb_params)

    model.fit(
        X_train,
        y_train,
        eval_set=[(X_val, y_val)],
        verbose=False,
    )

    val_preds = model.predict_proba(X_val)[:, 1]
    oof_preds[val_idx] = rankdata(val_preds)

    auc = roc_auc_score(y_val, val_preds)
    fold_scores.append(auc)

    print(f"Fold {fold} AUC: {auc:.5f}")

print("\n==============================")
print("Fold AUCs:", [f"{s:.5f}" for s in fold_scores])
print("Mean AUC :", np.mean(fold_scores))
print("OOF AUC  :", roc_auc_score(y, oof_preds))
print("==============================")


##### STACK FOLD 1 #####
Fold 1 AUC: 0.75610

##### STACK FOLD 2 #####
Fold 2 AUC: 0.75643

##### STACK FOLD 3 #####
Fold 3 AUC: 0.76022

##### STACK FOLD 4 #####
Fold 4 AUC: 0.76307

##### STACK FOLD 5 #####
Fold 5 AUC: 0.75704

Fold AUCs: ['0.75610', '0.75643', '0.76022', '0.76307', '0.75704']
Mean AUC : 0.7585728593018518
OOF AUC  : 0.7585639778208273


In [19]:
# SAVE GNN OOF
np.save(f"data/oof_preds_xgb_gnn_v{VER}",oof_preds)

# Compare to Ensemble

In [20]:
oof_xgb = np.load(f"data/oof_preds_xgb_v{LOAD}.npy")
print("XGB OOF AUC  :", float(roc_auc_score(author_features[TARGET].values, oof_xgb)))

oof_gnn = np.load(f"data/oof_preds_gnn_v{VER}.npy")
print("GNN OOF AUC  :", float(roc_auc_score(author_features[TARGET].values, oof_gnn)))

blend = (oof_xgb + oof_gnn)/2.
print("BLEND OOF AUC  :", float(roc_auc_score(author_features[TARGET].values, blend)))

oof_xgb_gnn = np.load(f"data/oof_preds_xgb_gnn_v{VER}.npy")
print("XGB w/ GNN emb  :", float(roc_auc_score(author_features[TARGET].values, oof_xgb_gnn)))

XGB OOF AUC  : 0.755080970524253
GNN OOF AUC  : 0.7556822341151668
BLEND OOF AUC  : 0.7576088669046419

XGB w/ GNN emb  : 0.7585639778208273
